In [1]:
import re
import numpy as np
import itertools
from collections import Counter

https://www.redblobgames.com/grids/hexagons/#neighbors

Using axial coordinates

In [2]:
E = 0
SE = 1
SW = 2
W = 3
NW = 4
NE = 5

In [3]:
direction_re = re.compile("(e)|(se)|(sw)|(w)|(nw)|(ne)")

def parse_input(inp):
    all_directions = []
    
    for l in inp.split('\n'):
        directions = []
        for m in direction_re.findall(l):
            ss = [s for s in m if s]
            assert len(ss) == 1

            d = ss.pop()
                        
            if d == 'e':
                n = E
            elif d == 'se':
                n = SE
            elif d == 'sw':
                n = SW
            elif d == 'w':
                n = W
            elif d == 'nw':
                n = NW
            elif d == 'ne':
                n = NE
            else:
                raise Exception('unknown direction ', d)

            directions.append(n)
    
        all_directions.append(directions)

    return all_directions

In [4]:
test_instructions = parse_input(open('./inputs/24_test').read())

In [5]:
instructions = parse_input(open('./inputs/24').read())

In [6]:
# axial coordinates
direction_vs = [
    # E
    [1, 0],
    #SE
    [0, 1],
    
    # SW
    [-1, 1],
    # W
    [-1, 0],
    
    # NW
    [0, -1],
    # NE
    [1, -1],
]

direction_vs = [np.array(d) for d in direction_vs]

BLACK = 0
WHITE = 1

In [7]:
def follow_instructions(instruction_list):
    max_length = max(len(ds) for ds in instruction_list) + 102
    
    d = 2 * max_length + 1
    floor = np.full((d,d), WHITE, dtype=np.short)
    
    reference_c = np.array([max_length,max_length])
    
    for instructions in instruction_list:
        current_c = reference_c

        for direction in instructions:        
            # move current_c in specified direction
            direction_v = direction_vs[direction]

            current_c = current_c + direction_v

        # flip where we landed
        floor[tuple(current_c)] = int(not floor[tuple(current_c)])

    print(Counter(floor.flatten()))
    
    return floor

In [8]:
test_floor = follow_instructions(test_instructions)

Counter({1: 61991, 0: 10})


In [9]:
floor = follow_instructions(instructions)

Counter({1: 61694, 0: 307})


In [10]:
def count_neighbors(floor, c):
    to_check = [c + d for d in direction_vs]

    c = Counter(floor[tuple(zip(*to_check))])
    
    return c

In [11]:
def follow_instructions2(floor, num_days=100):         
    current_floor = floor.copy()

    for day in range(num_days):
        next_floor = current_floor.copy()
        
        # iterate over all tile coordinates, 
        for tile_c, tile in np.ndenumerate(current_floor):
            # exclude outer perimeter to make bounds checking easier
            # just have to make sure board is padded enough
            if tile_c[0] == 0 or tile_c[0] == floor.shape[0] - 1 or tile_c[1] == 0 or tile_c[1] == floor.shape[1] - 1:
                continue
            
            # count neighbors in current_floor
            count = count_neighbors(current_floor, tile_c)
            tile_c = tuple(tile_c)
            
            if tile == BLACK and (count[BLACK] == 0 or count[BLACK] > 2):
                next_floor[tile_c] = WHITE
            elif tile == WHITE and count[BLACK] == 2:
                next_floor[tile_c] = BLACK

        current_floor = next_floor
        
        if (day + 1) % 10 == 0:
            print(day + 1)
            print(Counter(current_floor.flatten()))
            print()

In [12]:
test_floor.shape

(249, 249)

In [13]:
follow_instructions2(test_floor, 100)

10
Counter({1: 61964, 0: 37})

20
Counter({1: 61869, 0: 132})

30
Counter({1: 61742, 0: 259})

40
Counter({1: 61595, 0: 406})

50
Counter({1: 61435, 0: 566})

60
Counter({1: 61213, 0: 788})

70
Counter({1: 60895, 0: 1106})

80
Counter({1: 60628, 0: 1373})

90
Counter({1: 60157, 0: 1844})

100
Counter({1: 59793, 0: 2208})



In [14]:
follow_instructions2(floor, 100)

10
Counter({1: 61590, 0: 411})

20
Counter({1: 61390, 0: 611})

30
Counter({1: 61151, 0: 850})

40
Counter({1: 60882, 0: 1119})

50
Counter({1: 60521, 0: 1480})

60
Counter({1: 60206, 0: 1795})

70
Counter({1: 59706, 0: 2295})

80
Counter({1: 59308, 0: 2693})

90
Counter({1: 58810, 0: 3191})

100
Counter({1: 58214, 0: 3787})

